<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Task-description" data-toc-modified-id="Task-description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Task description</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Sentiment" data-toc-modified-id="Sentiment-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sentiment</a></span><ul class="toc-item"><li><span><a href="#Exploratory-analysis-for-sentiment" data-toc-modified-id="Exploratory-analysis-for-sentiment-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Exploratory analysis for sentiment</a></span></li><li><span><a href="#Sentiment-analysis" data-toc-modified-id="Sentiment-analysis-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Sentiment analysis</a></span></li></ul></li><li><span><a href="#Recommendations" data-toc-modified-id="Recommendations-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Recommendations</a></span><ul class="toc-item"><li><span><a href="#Train-test-split" data-toc-modified-id="Train-test-split-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Train-test split</a></span></li><li><span><a href="#Make-Turicreate-models" data-toc-modified-id="Make-Turicreate-models-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Make Turicreate models</a></span></li><li><span><a href="#Model-evaluation" data-toc-modified-id="Model-evaluation-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Model evaluation</a></span></li><li><span><a href="#Final-recommendation-system-(on-Turicreate)" data-toc-modified-id="Final-recommendation-system-(on-Turicreate)-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Final recommendation system (on Turicreate)</a></span></li></ul></li></ul></div>

# Task description
1. Using the dataset for Amazon Fine Food Reviews (https://www.kaggle.com/snap/amazon-fine-food-reviews, Reviews.csv) create a simple recommendation engine which recommends products to users based on their previous experience and rates. You can use either a user-based approach either product-based one.
To avoid excessive amount of workaround you can limit your model to score feature only (do not need to analyze review text).
<br><b>Deliverables</b>:
Your solution has to return three recommendations for every passed UserId and these recommended products have not match to products reviewed by user.
2. For Text field in dataset please create a mechanism to determine a review’s sentiment. Try to suggest alternative approach to using score as a sentiment label.
<br><br>NB: results have to be returned in .ipynb or .py format. 

# Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
import zipfile
from statistics import mean
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import pairwise_distances, pairwise_distances_chunked
from collections import Counter

# Load data

In [3]:
with zipfile.ZipFile('archive.zip') as z:
    df = pd.read_csv(z.open('Reviews.csv'))
df.shape

(568454, 10)

# Sentiment

## Exploratory analysis for sentiment

In [8]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [68]:
df.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,sentiment
count,568454.000000,568454.000000,568454.00000,568454.000000,5.684540e+05,568454.000000
mean,284227.500000,1.743817,2.22881,4.183199,1.296257e+09,0.296006
std,164098.679298,7.636513,8.28974,1.310436,4.804331e+07,0.253105
min,1.000000,0.000000,0.00000,1.000000,9.393408e+08,-0.974600
25%,142114.250000,0.000000,0.00000,4.000000,1.271290e+09,0.138038
50%,284227.500000,0.000000,1.00000,5.000000,1.311120e+09,0.297273
75%,426340.750000,2.000000,2.00000,5.000000,1.332720e+09,0.456933
max,568454.000000,866.000000,923.00000,5.000000,1.351210e+09,0.998300


In [16]:
for row in df[:10].itertuples():
    print(row.Text)

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.
If you are looking for the secret i

In [55]:
stop_words = set(stopwords.words("english"))
stop_words.update(['.', ',', '(', ')', '-', '...', "'", '``', "''"])

def remove_stop_words(tokenized_text:list[str]) -> list[str]:
    cleaned = [word for word in tokenized_text if word not in stop_words]
    return cleaned


lem = WordNetLemmatizer()

def lemmatize(tokenized_text:list[str]) -> list[str]:
    return [lem.lemmatize(word,"v").lower() for word in tokenized_text]

In [56]:
for row in df[:5].itertuples():
    text = row.Text
    print(text)
    tokens = [word for sent in sent_tokenize(text) for word in word_tokenize(sent)]
    tokens = remove_stop_words(tokens)
    tokens = lemmatize(tokens)
    print(tokens)
    print('----')

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
['i', 'buy', 'several', 'vitality', 'can', 'dog', 'food', 'products', 'find', 'good', 'quality', 'the', 'product', 'look', 'like', 'stew', 'process', 'meat', 'smell', 'better', 'my', 'labrador', 'finicky', 'appreciate', 'product', 'better']
----
Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
['product', 'arrive', 'label', 'jumbo', 'salted', 'peanuts', 'peanuts', 'actually', 'small', 'size', 'unsalted', 'not', 'sure', 'error', 'vendor', 'intend', 'represent', 'product', 'jumbo']
----
This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with n

## Sentiment analysis

In [58]:
sid = SentimentIntensityAnalyzer()
for row in df[:5].itertuples():
    text = row.Text
    for sentence in sent_tokenize(text):
        print(sentence, sid.polarity_scores(sentence))

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. {'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'compound': 0.6369}
The product looks more like a stew than a processed meat and it smells better. {'neg': 0.0, 'neu': 0.659, 'pos': 0.341, 'compound': 0.6901}
My Labrador is finicky and she appreciates this product better than  most. {'neg': 0.0, 'neu': 0.617, 'pos': 0.383, 'compound': 0.7351}
Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Not sure if this was an error or if the vendor intended to represent the product as "Jumbo". {'neg': 0.226, 'neu': 0.774, 'pos': 0.0, 'compound': -0.5664}
This is a confection that has been around a few centuries. {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
It is a light, pillowy citrus gelatin with nuts - in this case Filberts. {'neg': 0.173, 'neu': 0.827, 'pos': 0.0, 'compound': -0.3182

In [60]:
def sentiment(text:str) -> float:
    scores = [sid.polarity_scores(sentence)['compound'] for sentence in sent_tokenize(text)]
    return mean(scores)

In [61]:
df['sentiment'] = df['Text'].apply(sentiment)

In [64]:
df.corr()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,sentiment
Id,1.000000,0.001227,0.000770,0.010706,0.007912,0.005089
HelpfulnessNumerator,0.001227,1.000000,0.974689,-0.032590,-0.154818,-0.045123
HelpfulnessDenominator,0.000770,0.974689,1.000000,-0.097986,-0.173289,-0.077543
Score,0.010706,-0.032590,-0.097986,1.000000,-0.062760,0.470667
Time,0.007912,-0.154818,-0.173289,-0.062760,1.000000,-0.037347
sentiment,0.005089,-0.045123,-0.077543,0.470667,-0.037347,1.000000


# Recommendations

## Train-test split 

In [185]:
df['Time'].sort_values().reset_index().iloc[len(df)*2//3]

index        334326
Time     1326412800
Name: 378970, dtype: int64

In [6]:
(len(df[df['Time']<1326412800]), len(df[df['Time']>=1326412800]))

(378584, 189870)

In [4]:
df_train = df[df['Time']<1326412800][['UserId', 'ProductId', 'Score']]
df_test = df[df['Time']>=1326412800][['UserId', 'ProductId', 'Score']]

df_train['user_id'] = df_train['UserId'].factorize()[0]
df_train['product_id'] = df_train['ProductId'].factorize()[0]

## Make Turicreate models

In [12]:
import turicreate
train_data = turicreate.SFrame(df_train)
test_data = turicreate.SFrame(df_test)
popularity_model = turicreate.popularity_recommender.create(train_data, user_id='UserId', item_id='ProductId', target='Score')
item_similarity_model = turicreate.item_similarity_recommender.create(train_data, user_id='UserId', item_id='ProductId', target='Score')
factorization_model = turicreate.factorization_recommender.create(train_data, user_id='UserId', item_id='ProductId', target='Score')

Warning: Ignoring columns user_id, product_id;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 378584 observations with 175347 users and 53709 items.

Data prepared in: 0.3083s

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.282877

Final training RMSE: 0.531789

## Final recommendation system (on Turicreate)

In [1]:
def recommendation(UserId, n_recommendations=3):
    recommendations = {}
    reviewed_products = set(df[df.UserId==UserId]['ProductId'].unique())
    
    for model in popularity_model, factorization_model, item_similarity_model:
        for row in model.recommend(users=[UserId], k=10):
            recommendations[row['ProductId']] = max(recommendations.get(row['ProductId'], row['score']), row['score'])
    
    new_products = recommendations.keys() - reviewed_products
    recommendations = {key:recommendations[key] for key in new_products}
    top_recommendations = Counter(recommendations).most_common(n_recommendations)
    top_recommendations = [ProductId for ProductId, score in top_recommendations]
    return top_recommendations

In [ ]:
recommendation('A1D87F6ZCVE5NK')